In [1]:
import random
import itertools

In [10]:
class MarkovChain():
    def __init__(self, decay=1.0, state=2):
        self.decay = decay
        self.state = state
        self.reset()
        self.last_state = ''
        self.last_pred = ''

    def _create_matrix(self, order=1):
        def rec(s, depth, max_depth):
            if depth > max_depth:
                matrix[s] = {
                    'R': {'prob': 1/3, 'n_obs': 0},
                    'P': {'prob': 1/3, 'n_obs': 0},
                    'S': {'prob': 1/3, 'n_obs': 0}
                }
                return
            rec(s+'R', depth+1, max_depth)
            rec(s+'P', depth+1, max_depth)
            rec(s+'S', depth+1, max_depth)
        matrix = {}
        rec('', 0, self.state*2-1)
        return matrix

    def _update_matrix(self, pair, inp):
        for i in self.matrix[pair]:
            self.matrix[pair][i]['n_obs'] = self.decay * self.matrix[pair][i]['n_obs']

        self.matrix[pair][inp]['n_obs'] = self.matrix[pair][inp]['n_obs'] + 1
        
        n_total = 0
        for i in self.matrix[pair]:
            n_total += self.matrix[pair][i]['n_obs']
            
        for i in self.matrix[pair]:
            self.matrix[pair][i]['prob'] = self.matrix[pair][i]['n_obs'] / n_total            

    def reset(self):
        self.matrix = self._create_matrix()

    def predict(self):
        if len(self.last_state) < self.state*2:
            pred = random.choice(['R', 'P', 'S'])
        else:
            probs = self.matrix[self.last_state]
            if probs['R']['prob'] == probs['P']['prob'] == probs['S']['prob']:
                pred = random.choice(['R', 'P', 'S'])
            else:
                probs = self.matrix[self.last_state]
                decision = 'R'
                prob = probs['R']['prob']
                for i in {'P', 'S'}:
                    if probs[i]['prob'] > prob:
                        prob = probs[i]['prob']
                        decision = i
                beat = {'R':'P', 'P':'S', 'S':'R'}
                pred = beat[decision]

        self.last_pred = pred
        # return {'R':'rock', 'P':'paper', 'S':'scissors'}[pred]
        return pred
    
    def update(self, player_rps):
        # player_rps = {'rock':'R', 'paper':'P', 'scissors':'S'}[player_rps]
        self.last_state += player_rps + self.last_pred
        if len(self.last_state) == self.state * 2:
            self._update_matrix(self.last_state, player_rps)
        elif len(self.last_state) > self.state * 2:
            self.last_state = self.last_state[-self.state*2:]
            self._update_matrix(self.last_state, player_rps)

In [11]:
agent = MarkovChain(0.9)

In [12]:
agent.predict()

'R'

In [13]:
agent.update('R')

In [14]:
matrix = agent.matrix

In [18]:
print('--------------------------------------------------------------------')
for i in matrix:
    print(i, '|', matrix[i]['R']['prob'], '|', matrix[i]['P']['prob'], '|', matrix[i]['S']['prob'], '|')
    print('--------------------------------------------------------------------')

------------------------------------------------------------------------
RRRR | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |
------------------------------------------------------------------------
RRRP | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |
------------------------------------------------------------------------
RRRS | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |
------------------------------------------------------------------------
RRPR | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |
------------------------------------------------------------------------
RRPP | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |
------------------------------------------------------------------------
RRPS | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |
------------------------------------------------------------------------
RRSR | 0.3333333333333333 | 0.3333333333333333 | 0.3333333333333333 |